In [ ]:
# This tutorial is available for download as a Jupyter notebook



# Sensor Data Simulation with SensorSimulator

This tutorial demonstrates how to use the :class:`~calibrain.sensor_simulation.SensorSimulator` class
to generate synthetic MEG/EEG sensor measurements from brain source activity. The SensorSimulator 
projects source-level neural signals to sensor space using forward models and adds realistic noise,
creating controlled datasets for testing source localization algorithms, validating analysis 
pipelines, and benchmarking uncertainty quantification methods.


In [ ]:
# Authors: Mohammad Orabe  <m.orabe@icloud.com>
# License: AGPL-3.0 license
# Copyright the CaliBrain contributors.

## Background

This tutorial focuses on simulating sensor-level MEG/EEG measurements from
known source activity, which forms the bridge between source simulation and
inverse problem validation.

The SensorSimulator takes source time courses and projects them to sensor space
using leadfield matrices (forward models). It then adds controlled noise to create
realistic sensor measurements that can be used for algorithm testing and validation.
The class supports both fixed and free source orientations and provides precise
control over signal-to-noise ratios.

**Scope:** The CaliBrain framework implements a comprehensive 7-stage pipeline for
neuroimaging simulation and validation. This tutorial covers **sensor-level 
simulation** - the projection of neural activity from sources to sensors with
controlled noise addition. For generating source activity patterns, see the 
source simulation tutorial. For recovering sources from sensor data, see the 
source estimation tutorial.

```text
┌───────────────┐    ┌───────────────┐    ┏━━━━━━━━━━━━━━━┓    ┌───────────────┐
│   Leadfield   │ →  │    Source     │ →  ┃   ★ Sensor    ┃ →  │    Source     │
│ Construction  │    │  Simulation   │    ┃   Simulation  ┃    │  Estimation   │
└───────────────┘    └───────────────┘    ┗━━━━━━━━━━━━━━━┛    └───────────────┘
                                                                      ↓
                     ┌───────────────┐    ┌───────────────┐    ┌───────────────┐
                     │  Evaluation   │ ←  │ Visualization │ ←  │  Uncertainty  │
                     │               │    │               │    │   Estimation  │
                     └───────────────┘    └───────────────┘    └───────────────┘
```


## Mathematical Formulation

Sensor-level simulation projects sources to sensors via the forward model 
and adds realistic noise to create synthetic EEG/MEG data suitable for 
testing inverse methods and uncertainty estimation.

### Forward Projection

The sensor data $\mathbf{Y} \in \mathbb{R}^{M \times T}$ is generated by 
projecting source activity through a leadfield matrix and adding noise:

\begin{align}\mathbf{Y} = \mathbf{L} \mathbf{X} + \boldsymbol{\epsilon}\end{align}

where:

- $\mathbf{L} \in \mathbb{R}^{M \times N}$ is the leadfield matrix
- $\mathbf{X} \in \mathbb{R}^{N \times T}$ is the source activity matrix  
- $M$ is the number of sensors
- $N$ is the number of source locations (dipoles)
- $T$ is the number of time points

The leadfield provides the mapping from source current to sensor measurements,
with units typically in volts per ampere-meter (V/Am) for EEG and tesla per 
ampere-meter (T/Am) for MEG.

### Dipole Orientations

Source activity can be modeled with different orientation constraints:

**Fixed orientations**: Sources are constrained to be normal to the cortical 
surface, reducing the source space dimensionality:

\begin{align}\mathbf{L}_{\text{fixed}} \in \mathbb{R}^{M \times N_{\text{vertices}}}\end{align}

**Free orientations**: Sources can point in any direction, typically represented 
as three orthogonal components per vertex:

\begin{align}\mathbf{L}_{\text{free}} \in \mathbb{R}^{M \times 3N_{\text{vertices}}}\end{align}

The CaliBrain framework supports both orientation models through the 
:class:`~calibrain.leadfield_builder.LeadfieldBuilder` class.

### Noise Model and Alpha-SNR Framework

Sensor noise is modeled as additive white Gaussian noise with precise control
through the Alpha-SNR parameter $\alpha_{\text{SNR}}$.

**Classical Signal-to-Noise Ratio**

The traditional signal-to-noise ratio is defined as the ratio of signal power
to noise power:

\begin{align}\text{SNR}_{\text{linear}} = \frac{P_{\text{signal}}}{P_{\text{noise}}} = \frac{\|\mathbf{Y}_{\text{clean}}\|_F^2}{\|\boldsymbol{\epsilon}\|_F^2}\end{align}

where the signal and noise powers are defined as:

\begin{align}P_{\text{signal}} = \|\mathbf{Y}_{\text{clean}}\|_F^2 = \|\mathbf{L}\mathbf{X}\|_F^2 = \sum_{m=1}^{M} \sum_{t=1}^{T} (\mathbf{Y}_{\text{clean}})_{m,t}^2\end{align}

\begin{align}P_{\text{noise}} = \|\boldsymbol{\epsilon}\|_F^2 = \sum_{m=1}^{M} \sum_{t=1}^{T} \epsilon_{m,t}^2\end{align}

The Frobenius norm $\|\cdot\|_F$ computes the total squared magnitude across
all sensor measurements and time points, providing a scalar measure of overall
signal or noise energy.

In decibel (dB) scale:

\begin{align}\text{SNR}_{\text{dB}} = 10 \log_{10}(\text{SNR}_{\text{linear}})\end{align}

**Alpha-SNR Framework**

The Alpha-SNR parameter $\alpha_{\text{SNR}} \in [0, 1]$ provides an intuitive
representation where:

\begin{align}\alpha_{\text{SNR}} = \frac{P_{\text{signal}}}{P_{\text{signal}} + P_{\text{noise}}}\end{align}

This formulation offers several advantages:

- $\alpha_{\text{SNR}} = 1.0$: Pure signal, no noise added
- $\alpha_{\text{SNR}} = 0.5$: Equal signal and noise power  
- $\alpha_{\text{SNR}} = 0.0$: Pure noise, no signal
- Linear interpolation between extreme cases

**Noise Scaling Factor**

The noise scaling factor $\eta$ is derived to achieve the target Alpha-SNR:

\begin{align}\eta = \sqrt{\frac{1 - \alpha_{\text{SNR}}}{\alpha_{\text{SNR}}}} \cdot \frac{\|\mathbf{Y}_{\text{clean}}\|_F}{\|\boldsymbol{\epsilon}_{\text{base}}\|_F}\end{align}

where $\boldsymbol{\epsilon}_{\text{base}} \sim \mathcal{N}(0, \mathbf{I})$ is the base
white noise matrix with unit variance.

**SNR Conversions**

The relationships between different SNR representations are:

\begin{align}\text{SNR}_{\text{linear}} = \frac{\alpha_{\text{SNR}}}{1 - \alpha_{\text{SNR}}}\end{align}

\begin{align}\alpha_{\text{SNR}} = \frac{\text{SNR}_{\text{linear}}}{1 + \text{SNR}_{\text{linear}}}\end{align}

\begin{align}\text{SNR}_{\text{dB}} = 10 \log_{10}\left(\frac{\alpha_{\text{SNR}}}{1 - \alpha_{\text{SNR}}}\right)\end{align}

**Conversion Reference Table**

Common Alpha-SNR values and their classical SNR equivalents:

========== ============ =======
Alpha-SNR  Linear SNR   dB SNR
========== ============ =======
0.95       19.00        12.79
0.80       4.00         6.02
0.50       1.00         0.00
0.20       0.25         -6.02
0.05       0.05         -12.79
========== ============ =======

The noise model assumes homoscedastic (uniform variance) and uncorrelated 
(white) Gaussian noise across all sensors and time points.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from pathlib import Path

## The SensorSimulator Class

The :class:`~calibrain.sensor_simulation.SensorSimulator` class is located in the 
``calibrain/sensor_simulation.py`` module and serves as the core component 
for projecting source activity to sensor measurements with controlled noise.



### Class Methods

The :class:`~calibrain.sensor_simulation.SensorSimulator` class consists of three core methods:
   - :meth:`~calibrain.sensor_simulation.SensorSimulator._project_sources_to_sensors`: Internal method for forward projection using leadfield matrices
   - :meth:`~calibrain.sensor_simulation.SensorSimulator._add_noise`: Internal method for adding controlled Gaussian noise to clean signals
   - :meth:`~calibrain.sensor_simulation.SensorSimulator.simulate`: Main public method for multi-trial sensor simulation. This is the main method that should be used as it wraps the core functionality.

The :class:`~calibrain.sensor_simulation.SensorSimulator` class is part of the CaliBrain package and can be imported directly as follows:




In [ ]:
from calibrain import SensorSimulator

### Basic Usage Example
The :class:`~calibrain.sensor_simulation.SensorSimulator` class expects a logger for tracking operations. However if we do not provide one, a default logger will be used.
The :meth:`~calibrain.sensor_simulation.SensorSimulator.simulate` function requires source data, leadfield matrices, and simulation parameters.

Quick demonstration with minimal setup:



In [ ]:
demo_sensor_simulator = SensorSimulator()   # use default logger

# We can always use the CaliBrain's utility function :func:`~calibrain.utils.inspect_object` to explore any 
# class structure and understand its available attributes and methods:

from calibrain import utils
from calibrain.utils import inspect_object

class_info = inspect_object(demo_sensor_simulator, show_private=False)

## Quick Demo with Random Data

Let's start with a simple demonstration using random source activity and a 
synthetic leadfield matrix to understand the core concepts:



In [ ]:
# Create random source time courses (3 sources, 100 time points, 2 trials)
n_sources_demo = 3
n_sensors_demo = 5
n_times_demo = 100
n_trials_demo = 2

# Random source activity for two trials: sources × time
np.random.seed(16)
x_demo_trial1 = np.random.randn(n_sources_demo, n_times_demo) # assuming (Am)

np.random.seed(84)  # Different seed for trial 2
x_demo_trial2 = np.random.randn(n_sources_demo, n_times_demo) # assuming (Am)

x_trials_demo = np.array([x_demo_trial1, x_demo_trial2])

# Random leadfield matrix: sensors × sources  
# Scale to get realistic sensor voltages (V) from source currents (Am)
# For EEG-like measurements: ~1e-6 V/Am (microVolts per Ampere·meter)
L_demo = np.random.randn(n_sensors_demo, n_sources_demo) * 1e-6  # V/Am 

print(f"  - Source activity shape: {x_trials_demo.shape}") # (trials × sources × time)
print(f"  - Leadfield matrix shape: {L_demo.shape}") # (sensors × sources)

Run the simple sensor simulation:



In [ ]:
y_clean_demo, y_noisy_demo, noise_demo, noise_var_demo = demo_sensor_simulator.simulate(
    x_trials=x_trials_demo,
    L=L_demo,
    orientation_type="fixed",
    alpha_SNR=0.7,  # 70% signal, 30% noise
    n_trials=n_trials_demo,
    global_seed=42
)

print(f"\nDemo Results:")
print(f"  - Clean sensor data shape: {y_clean_demo.shape} (trials × sensors × time)")
print(f"  - Noisy sensor data shape: {y_noisy_demo.shape}")

<div class="alert alert-info"><h4>Note</h4><p>Sensor values are in automatically in Tesla (T) for MEG.
   We need to set the units manually to volts (V) for EEG. This will be useful for 
   visualization. Usually we can derive the correct units from the forward operator.</p></div>



In [ ]:
from mne.io.constants import FIFF

print(demo_sensor_simulator.sensor_units) # -> Tesla (T)

demo_sensor_simulator.sensor_units = FIFF.FIFF_UNIT_V
print(demo_sensor_simulator.sensor_units) # -> Volts (V)

## Realistic Simulation Pipeline

Now let's move to a more realistic example using the full CaliBrain pipeline
with anatomically-based leadfield matrices and physiologically-plausible 
source activity patterns.



Configure logging to see simulation progress



In [ ]:
logging.basicConfig(
    level=logging.INFO, # Set to logging.DEBUG for more detailed output
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("SensorSimulator")

### Step 1: Load Leadfield Matrix

Next, we need a leadfield matrix to project sources to sensors:



In [ ]:
from calibrain import LeadfieldBuilder
from calibrain.utils import get_data_path

data_path = get_data_path() # Get the default calibrain/data path

# Initialize leadfield builder
leadfield_builder = LeadfieldBuilder(
    leadfield_dir=data_path,
    logger=logger
)

# Load leadfield matrix for a standard subject
L = leadfield_builder.get_leadfield(
    subject="fsaverage",
    orientation_type="fixed",
    retrieve_mode="load"
)

n_sensors, n_sources = L.shape

print(f"\nLeadfield matrix:")
print(f"  - Contains {L.shape[0]} sensors and {L.shape[1]} sources")
print(f"  - Sensor units: {leadfield_builder.sensor_units}")

### Step 2: Generate Source Activity

First, we need source time courses to project to sensors. We'll use the
SourceSimulator to create realistic ERP-like source activity:



In [ ]:
from calibrain import SourceSimulator

# Configure ERP parameters for source simulation
erp_config = {
    "tmin": -0.5,                # Start time of epoch (seconds before stimulus)
    "tmax": 0.5,                 # End time of epoch (seconds after stimulus)
    "stim_onset": 0,             # Stimulus onset time (relative to epoch start)
    "sfreq": 250,                # Sampling frequency in Hz
    "fmin": 1,                   # Minimum frequency for bandpass filter (Hz)
    "fmax": 5,                   # Maximum frequency for bandpass filter (Hz)
    "amplitude": 50.0,           # Amplitude scaling factor (nAm)
    "random_erp_timing": True,   # Whether to randomize ERP onset and duration
    "erp_min_length": 80         # Minimum ERP duration in samples
}

# Create source simulator and generate activity
source_simulator = SourceSimulator(ERP_config=erp_config, logger=logger)

# Simulation parameters for source activity
source_params = {
    "orientation_type": "fixed",  # Source orientation type
    "n_sources": n_sources,       # Total number of source locations (fsaverage)
    "nnz": 5,                     # Number of active sources per trial
    "n_trials": 3,                # Number of trials to simulate
    "global_seed": 42             # Seed for reproducibility
}

# Generate source time courses
x_trials, x_active_indices_trials = source_simulator.simulate(**source_params)

print(f"Generated source activity:")
print(f"  - Shape: {x_trials.shape} (trials x sources x time)")
print(f"  - Active sources per trial: {[len(indices) for indices in x_active_indices_trials]}")
print(f"  - Source units: {source_simulator.source_units}")

<div class="alert alert-info"><h4>Note</h4><p>For an in-depth guide to the :class:`~calibrain.source_simulation.SourceSimulator` class and advanced visualization techniques, refer to the `tut-source-simulation` tutorial.</p></div>



### Step 3: Configure Sensor Simulation

Now we can create the SensorSimulator and project sources to sensors:



In [ ]:
# Create sensor simulator
sensor_simulator = SensorSimulator(logger=logger)
print(f"- Default sensor units: {sensor_simulator.sensor_units}")

<div class="alert alert-info"><h4>Note</h4><p>Default sensor units are set to Tesla (T) which corresponds to FIFF.FIFF_UNIT_T
   in MNE notation for MEG magnetometer sensors. Usually, the units will be
   automatically updated based on the leadfield matrix sensor type during simulation.</p></div>



In [ ]:
sensor_simulator.sensor_units = leadfield_builder.sensor_units

## Visualization with CaliBrain

The CaliBrain :class:`~calibrain.visualization.Visualizer` provides sophisticated plotting capabilities for
sensor data analysis



In [ ]:
from calibrain import Visualizer

# Create visualizer instance with output directory
save_path = Path("tutorial_results")
save_path.mkdir(exist_ok=True)

viz = Visualizer(base_save_path=str(save_path), logger=logger)

<div class="alert alert-info"><h4>Note</h4><p>For an in-depth guide to the :class:`~calibrain.visualization.Visualizer` class and advanced visualization techniques, refer to the `_tut-visualization` tutorial.</p></div>



Configure sensor simulation parameters



In [ ]:
sensor_simulation_params = {
    "x_trials": x_trials,                    # Source time courses from Step 1
    "L": L,                                  # Leadfield matrix from Step 2
    "orientation_type": "fixed",             # Must match source simulation
    "alpha_SNR": 0.3,                       # Signal-to-noise ratio (30% signal, 70% noise)
    "n_trials": 3,                          # Number of trials (must match source trials)
    "global_seed": 42                       # Seed for reproducible noise
}

# Execute sensor simulation
y_clean_trials, y_noisy_trials, noise_trials, noise_var_trials = sensor_simulator.simulate(
    **sensor_simulation_params
)

Let's inspect the sensor simulation results:



In [ ]:
print(f"  - Clean sensor data shape: {y_clean_trials.shape} (trials x sensors x time)")
print(f"  - Noisy sensor data shape: {y_noisy_trials.shape} (trials x sensors x time)")
print(f"  - Noise data shape: {noise_trials.shape} (trials x sensors x time)")
print(f"  - Noise variance per trial: {noise_var_trials.shape} (trials x sensors)")

### Plot Sensor Signals for All Trials

First, let's visualize the clean and noisy sensor signals:



Plot clean sensor signals (concatenated)



In [ ]:
viz.plot_sensor_signals(
    ERP_config=erp_config,
    y_trials=y_clean_trials,                     # Only plot clean signals
    # trial_idx = 0,
    # channels=[0, 10],                          # or "all"
    units=sensor_simulator.sensor_units,
    mode="concatenate",                          # or "stack"
    title="Sensor Signals (All trials concatenated)",
    save_dir="data_simulation",
    file_name="sensor_concatenate_trials_clean",
    show=True
)

Plot sensors (all trials) with selected channels: y_noisy



In [ ]:
viz.plot_sensor_signals(
    ERP_config=erp_config,
    y_trials=y_noisy_trials,                    # Only noisy signals
    # trial_idx = 0,
    channels="all",                              # or "all"
    units=sensor_simulator.sensor_units,
    mode="stack",                                # or "stack"
    title="Sensor Signals (All Trials stacked)",
    save_dir="data_simulation",
    file_name="sensor_stack_trials_noisy",
    show=True
)

Plot noisy sensor signals (concatenated)



In [ ]:
viz.plot_sensor_signals(
    ERP_config=erp_config,
    y_trials=y_noisy_trials,
    # trial_idx = 0,
    # channels=[0, 10],                           # or "all"
    units=sensor_simulator.sensor_units,
    mode="concatenate",                           # or "stack"
    title="Sensor Signals (All trials concatenated)",
    save_dir="data_simulation",
    file_name="sensor_concatenate_trials_noisy",
    show=True
)

<div class="alert alert-info"><h4>Note</h4><p>The visualization automatically handles unit conversion and scaling for optimal
   display. MEG sensor measurements are typically displayed in femtoTesla (fT) and
   EEG measurements in microVolts (µV).</p></div>



## Advanced SNR Configuration

The SensorSimulator supports various SNR configurations to simulate different
experimental conditions. Let's explore different noise levels:



### High SNR (Clean Signals)

Configuration for high signal-to-noise ratio,



In [ ]:
high_snr_params = {
    "x_trials": x_trials,
    "L": L,
    "orientation_type": "fixed",
    "alpha_SNR": 0.9,                       # 90% signal, 10% noise
    "n_trials": 3,
    "global_seed": 100                      # Different seed for variety
}

y_clean_high, y_noisy_high, noise_high, noise_var_high = sensor_simulator.simulate(**high_snr_params)

print(f"\nHigh SNR Results (α_SNR = 0.9):")
print(f"  - Signal range: [{y_clean_high.min():.2e}, {y_clean_high.max():.2e}]")
print(f"  - Noise range: [{noise_high.min():.2e}, {noise_high.max():.2e}]")

### Low SNR (Noisy Signals)  

Configuration for low signal-to-noise ratio, typical of single-trial data
or challenging recording conditions:



In [ ]:
low_snr_params = {
    "x_trials": x_trials,
    "L": L,
    "orientation_type": "fixed",
    "alpha_SNR": 0.1,                       # 10% signal, 90% noise
    "n_trials": 3,
    "global_seed": 200                      # Different seed for variety
}

y_clean_low, y_noisy_low, noise_low, noise_var_low = sensor_simulator.simulate(**low_snr_params)

print(f"\nLow SNR Results (α_SNR = 0.1):")
print(f"  - Signal range: [{y_clean_low.min():.2e}, {y_clean_low.max():.2e}]")
print(f"  - Noise range: [{noise_low.min():.2e}, {noise_low.max():.2e}]")

### Visualize SNR Comparisons

Create comparative visualizations to highlight the effect of different SNR levels:



In [ ]:
# High SNR visualization
viz.plot_sensor_signals(
    ERP_config=erp_config,
    y_trials=y_noisy_high,
    units=leadfield_builder.sensor_units,
    trial_idx=0,
    title="High SNR (α_SNR = 0.9) - Clean vs Noisy",
    save_dir="snr_comparison",
    file_name="high_snr_comparison",
    show=True
)

Low SNR visualization



In [ ]:
viz.plot_sensor_signals(
    ERP_config=erp_config,
    y_trials=y_noisy_low,
    units=leadfield_builder.sensor_units,
    trial_idx=0,
    title="Low SNR (α_SNR = 0.1) - Clean vs Noisy",
    save_dir="snr_comparison",
    file_name="low_snr_comparison",
    show=True
)

## Integration with CaliBrain Pipeline

The SensorSimulator is designed to work seamlessly with other CaliBrain
components in a complete neuroimaging simulation pipeline.



**Example Pipeline Code**

Here's a conceptual example of how :class:`~calibrain.sensor_simulation.SensorSimulator` integrates with
other CaliBrain components:



In [ ]:
pipeline_example = '''
# Complete CaliBrain simulation pipeline example

from calibrain import (
    LeadfieldBuilder, SourceSimulator, SensorSimulator, 
    SourceEstimator, UncertaintyEstimator, MetricEvaluator, Visualizer
)

# 1. Build leadfield matrix
leadfield_builder = LeadfieldBuilder(...)
L = leadfield_builder.get_leadfield(
    subject="fsaverage", 
    orientation_type="fixed"
)

# 2. Simulate source activity  
source_simulator = SourceSimulator(ERP_config=erp_config)
x_trials, x_active_indices = source_simulator.simulate(
    orientation_type="fixed",
    n_sources=L.shape[1],  # Match leadfield dimensions
    nnz=5,
    n_trials=10
)

# 3. Simulate sensor measurements
sensor_simulator = SensorSimulator()
y_clean, y_noisy, noise, noise_var = sensor_simulator.simulate(
    x_trials=x_trials,
    L=L,
    alpha_SNR=0.3,
    n_trials=10
)

# 4. Estimate sources
source_estimator = SourceEstimator(solver="gamma_map")
source_estimator.fit(L, y_noisy[0])
x_hat, x_hat_indices, posterior_cov = source_estimator.predict(y_noisy[0])

# 5. Estimate uncertainty (-> credible intervals)
uncertainty_estimator = UncertaintyEstimator()
ci_lower, ci_upper, _, empirical_coverage = \\
    uncertainty_estimator.get_confidence_intervals_data(
        x=x_trials[0],
        x_hat=x_hat,
        posterior_cov=posterior_cov,
        orientation_type="fixed"
    )

# 6. Evaluate performance
metric_evaluator = MetricEvaluator()
metrics = metric_evaluator.evaluate(
    x_true=x_trials[0],
    x_hat=x_hat,
    active_indices_true=x_active_indices[0],
    active_indices_hat=x_hat_indices
)

# 7. Visualize results
visualizer = Visualizer()
visualizer.plot_source_comparison(x_trials, x_hat, x_active_indices)
'''

print(f"\nPipeline Integration Example:")
print(f"See 'examples/run_experiments.py' and 'calibrain/benchmark.py'")
print(f"for complete working examples of integrated simulations.")

print(f"\nSensor Simulation Tutorial Complete!")
print(f"Generated sensor data with shape: {y_noisy_trials.shape}")
print(f"Tutorial results saved to: {save_path}")